In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('/kaggle/input/multimodal-sentiment/Multimodal_dataset_assignment3/labels.csv')

In [4]:
df.isna().sum()

Unnamed: 0             0
image_name             0
text_ocr             161
text_corrected         5
humour                 0
sarcasm                0
offensive              0
motivational           0
overall_sentiment      0
dtype: int64

In [5]:
df

,Unnamed: 0,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral
...,...,...,...,...,...,...,...,...,...
6987,6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,Tuesday is Mardi Gras Wednesday is Valentine's...,very_funny,twisted_meaning,very_offensive,motivational,neutral
6988,6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,funny,twisted_meaning,not_offensive,not_motivational,neutral
6989,6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,funny,general,slight,not_motivational,positive
6990,6990,image_6991.jpg,When I VERY have time is a fantasy No one has ...,When I have time is a fantasy. no one has time...,not_funny,twisted_meaning,not_offensive,motivational,very_positive


In [6]:
df.columns

Index(['Unnamed: 0', 'image_name', 'text_ocr', 'text_corrected', 'humour',
       'sarcasm', 'offensive', 'motivational', 'overall_sentiment'],
      dtype='object')

In [7]:
df = df.drop(columns=['Unnamed: 0','text_ocr'])

In [8]:
df = df.dropna()
df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral


In [9]:
import re

def clean_text(text):
    # Remove @usernames
    text = re.sub(r'@\w+', '', text)
    
    # Remove emails
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove all symbols except spaces and alphanumeric characters
    text = re.sub(r'[^a-z0-9\s]', '', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

In [10]:
df1 = df.copy()

In [11]:
df1['text_corrected'] = df1['text_corrected'].apply(clean_text)

In [12]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
def remove_stopwords(text, custom_stopwords=None):
    # Load default stopwords
    stop_words = set(stopwords.words('english'))
    stop_words.update({'follow'})

    # Remove stopwords
    filtered_text = ' '.join(word for word in text.split() if word not in stop_words)
    
    return filtered_text

In [14]:
df1['text_corrected'] = df1['text_corrected'].apply(remove_stopwords)

In [15]:
df1

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,look friend lightyear sohalikut trend play 10 ...,hilarious,general,not_offensive,not_motivational,very_positive
1,image_2.jpeg,best 10 yearchallenge completed less 4 years k...,not_funny,general,not_offensive,motivational,very_positive
2,image_3.JPG,sam thorne saw everyone posting 2009 vs 2019 p...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,image_4.png,10 year challenge sweet dee edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,image_5.png,10 year challenge filter 47 hilarious 10 year ...,hilarious,very_twisted,very_offensive,not_motivational,neutral
...,...,...,...,...,...,...,...
6987,image_6988.jpg,tuesday mardi gras wednesday valentines friday...,very_funny,twisted_meaning,very_offensive,motivational,neutral
6988,image_6989.jpg,must watch movies 2017 iti chennai memes maana...,funny,twisted_meaning,not_offensive,not_motivational,neutral
6989,image_6990.png,less talking planning soda junk food complaini...,funny,general,slight,not_motivational,positive
6990,image_6991.jpg,time fantasy one time unless make time arhtist...,not_funny,twisted_meaning,not_offensive,motivational,very_positive


In [16]:
df1['overall_sentiment'] = df1['overall_sentiment'].replace({'very_positive': 2, 'positive': 2, 'negative': 0, 'very_negative': 0, 'neutral': 1})
df1['overall_sentiment'].value_counts()

<ipython-input-16-7bff3999a30c>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['overall_sentiment'] = df1['overall_sentiment'].replace({'very_positive': 2, 'positive': 2, 'negative': 0, 'very_negative': 0, 'neutral': 1})


overall_sentiment
2    4156
1    2200
0     631
Name: count, dtype: int64

In [17]:
df1['humour_sentiment'] = df1['humour'].replace({'very_funny': 1, 'funny': 1, 'hilarious': 1, 'not_funny': 0})
df1['sarcasm_sentiment'] = df1['sarcasm'].replace({'very_twisted': 1, 'twisted_meaning': 1, 'general': 1, 'not_sarcastic': 0})
df1['offensive_sentiment'] = df1['offensive'].replace({'very_offensive': 1, 'hateful_offensive': 1, 'slight': 1, 'not_offensive': 0})

<ipython-input-17-d2289acd37cc>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['humour_sentiment'] = df1['humour'].replace({'very_funny': 1, 'funny': 1, 'hilarious': 1, 'not_funny': 0})
<ipython-input-17-d2289acd37cc>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['sarcasm_sentiment'] = df1['sarcasm'].replace({'very_twisted': 1, 'twisted_meaning': 1, 'general': 1, 'not_sarcastic': 0})
<ipython-input-17-d2289acd37cc>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future versio

In [18]:
df1.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment,humour_sentiment,sarcasm_sentiment,offensive_sentiment
0,image_1.jpg,look friend lightyear sohalikut trend play 10 ...,hilarious,general,not_offensive,not_motivational,2,1,1,0
1,image_2.jpeg,best 10 yearchallenge completed less 4 years k...,not_funny,general,not_offensive,motivational,2,0,1,0
2,image_3.JPG,sam thorne saw everyone posting 2009 vs 2019 p...,very_funny,not_sarcastic,not_offensive,not_motivational,2,1,0,0
3,image_4.png,10 year challenge sweet dee edition,very_funny,twisted_meaning,very_offensive,motivational,2,1,1,1
4,image_5.png,10 year challenge filter 47 hilarious 10 year ...,hilarious,very_twisted,very_offensive,not_motivational,1,1,1,1


In [19]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import ViTImageProcessor, ViTModel
from PIL import Image, ImageFile
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.decomposition import PCA
import torch.optim as optim
import numpy as np
import os

In [20]:
TEXT_MODEL_NAME = 'distilbert-base-uncased'
IMAGE_MODEL_NAME = 'google/vit-base-patch16-224'
MAX_LENGTH = 128
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 2e-5
PCA_COMPONENTS = 256  # Reduce feature dimensions
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
text_tokenizer = DistilBertTokenizer.from_pretrained(TEXT_MODEL_NAME)
image_processor = ViTImageProcessor.from_pretrained(IMAGE_MODEL_NAME)
text_model = DistilBertModel.from_pretrained(TEXT_MODEL_NAME).to(DEVICE)
image_model = ViTModel.from_pretrained(IMAGE_MODEL_NAME).to(DEVICE)

# Freeze transformers to avoid recomputation
text_model.eval()
image_model.eval()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTModel(
  (embeddings): ViTEmbeddings(
    (patch_embeddings): ViTPatchEmbeddings(
      (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): ViTEncoder(
    (layer): ModuleList(
      (0-11): 12 x ViTLayer(
        (attention): ViTSdpaAttention(
          (attention): ViTSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
          (output): ViTSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (intermediate): ViTIntermediate(
          (dense): Linear(in_features=768, out_features=3072, bias=True)
          (intermediate_act_fn): GELUAct

In [22]:
def preprocess_data(dataframe):
    texts = dataframe['text_corrected'].tolist()
    image_dir = "/kaggle/input/multimodal-sentiment/Multimodal_dataset_assignment3/images"
    image_paths = dataframe['image_name'].apply(lambda name: os.path.join(image_dir, name))
    # image_paths = dataframe['image_path'].tolist()
    labels = torch.tensor(dataframe[['humour_sentiment', 'sarcasm_sentiment', 'offensive_sentiment']].values, dtype=torch.float)
    
    text_features, image_features = [], []
    
    with torch.no_grad():
        for text, image_path in zip(texts, image_paths):
            text_inputs = text_tokenizer(text, max_length=MAX_LENGTH, padding='max_length', truncation=True, return_tensors='pt').to(DEVICE)
            text_output = text_model(**text_inputs).last_hidden_state[:, 0, :].cpu().numpy()

            ImageFile.LOAD_TRUNCATED_IMAGES = True
            image = Image.open(image_path).convert('RGB')
            image_inputs = image_processor(images=image, return_tensors='pt').to(DEVICE)
            image_output = image_model(**image_inputs).last_hidden_state[:, 0, :].cpu().numpy()
            
            text_features.append(text_output.squeeze())
            image_features.append(image_output.squeeze())
    
    return np.array(text_features), np.array(image_features), labels

In [23]:
class MemeDataset(Dataset):
    def __init__(self, text_features, image_features, labels):
        self.text_features = text_features
        self.image_features = image_features
        self.labels = labels

    def __len__(self):
        return len(self.text_features)

    def __getitem__(self, idx):
        return {
            'text_features': torch.tensor(self.text_features[idx], dtype=torch.float),
            'image_features': torch.tensor(self.image_features[idx], dtype=torch.float),
            'labels': self.labels[idx]
        }


In [24]:
class MultimodalModel(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc_humor = nn.Linear(input_dim, 1)
        self.fc_sarcasm = nn.Linear(input_dim, 1)
        self.fc_offensive = nn.Linear(input_dim, 1)
        self.dropout = nn.Dropout(0.1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text_features, image_features):
        combined = torch.cat([text_features, image_features], dim=1)
        combined = self.dropout(combined)
        
        humor = self.sigmoid(self.fc_humor(combined))
        sarcasm = self.sigmoid(self.fc_sarcasm(combined))
        offensive = self.sigmoid(self.fc_offensive(combined))
        
        return torch.cat([humor, sarcasm, offensive], dim=1)


In [25]:
def evaluate(model, loader, device):
    model.eval()
    humor_preds, humor_labels = [], []
    sarcasm_preds, sarcasm_labels = [], []
    offensive_preds, offensive_labels = [], []
    
    with torch.no_grad():
        for batch in loader:
            text_features = batch['text_features'].to(device)
            image_features = batch['image_features'].to(device)
            labels = batch['labels'].cpu().numpy()
            
            outputs = model(text_features, image_features).cpu().numpy()
            preds = (outputs >= 0.5).astype(int)
            
            humor_preds.extend(preds[:, 0])
            sarcasm_preds.extend(preds[:, 1])
            offensive_preds.extend(preds[:, 2])
            
            humor_labels.extend(labels[:, 0])
            sarcasm_labels.extend(labels[:, 1])
            offensive_labels.extend(labels[:, 2])
    
    metrics = {
        'humor_f1': f1_score(humor_labels, humor_preds, average='macro'),
        'sarcasm_f1': f1_score(sarcasm_labels, sarcasm_preds, average='macro'),
        'offensive_f1': f1_score(offensive_labels, offensive_preds, average='macro')
    }
    metrics['avg_f1'] = sum(metrics.values()) / 3
    return metrics

In [27]:
from sklearn.model_selection import train_test_split
# Split dataset
train_df, test_df = train_test_split(df1, test_size=0.2, random_state=42)

train_text, train_image, train_labels = preprocess_data(train_df)
val_text, test_image, test_labels = preprocess_data(test_df)



/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [28]:
pca = PCA(n_components=PCA_COMPONENTS)
train_text = pca.fit_transform(train_text)
train_image = pca.fit_transform(train_image)
val_image = pca.transform(test_image)
val_text = pca.transform(val_text)

In [29]:
train_dataset = MemeDataset(train_text, train_image, train_labels)
val_dataset = MemeDataset(val_text, val_image, test_labels)

In [30]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

In [31]:
model = MultimodalModel(input_dim=PCA_COMPONENTS * 2).to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCELoss()

In [32]:
model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for batch in train_loader:
            optimizer.zero_grad()
            
            text_features = batch['text_features'].to(DEVICE)
            image_features = batch['image_features'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)
            
            outputs = model(text_features, image_features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
    val_metrics = evaluate(model, val_loader, DEVICE)
    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"Train Loss: {total_loss/len(train_loader):.4f}")
    print(f"Val F1 Scores - Humor: {val_metrics['humor_f1']:.4f}, "
              f"Sarcasm: {val_metrics['sarcasm_f1']:.4f}, "
              f"Offensive: {val_metrics['offensive_f1']:.4f}")
    print(f"Avg F1: {val_metrics['avg_f1']:.4f}\n")
val_metrics = evaluate(model, val_loader, DEVICE)
checkpoint = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss,
                'val_f1': val_metrics['avg_f1']
                }

torch.save(checkpoint, 'checkpoint_TaskB.pth')

Epoch 1/10
Train Loss: 0.7269
Val F1 Scores - Humor: 0.3741, Sarcasm: 0.4001, Offensive: 0.4584
Avg F1: 0.4109

Epoch 2/10
Train Loss: 0.7211
Val F1 Scores - Humor: 0.3706, Sarcasm: 0.4018, Offensive: 0.4603
Avg F1: 0.4109

Epoch 3/10
Train Loss: 0.7188
Val F1 Scores - Humor: 0.3669, Sarcasm: 0.4044, Offensive: 0.4610
Avg F1: 0.4107

Epoch 4/10
Train Loss: 0.7166
Val F1 Scores - Humor: 0.3629, Sarcasm: 0.4072, Offensive: 0.4634
Avg F1: 0.4111

Epoch 5/10
Train Loss: 0.7143
Val F1 Scores - Humor: 0.3588, Sarcasm: 0.4068, Offensive: 0.4680
Avg F1: 0.4112

Epoch 6/10
Train Loss: 0.7123
Val F1 Scores - Humor: 0.3540, Sarcasm: 0.4067, Offensive: 0.4707
Avg F1: 0.4105

Epoch 7/10
Train Loss: 0.7102
Val F1 Scores - Humor: 0.3512, Sarcasm: 0.4085, Offensive: 0.4744
Avg F1: 0.4114

Epoch 8/10
Train Loss: 0.7081
Val F1 Scores - Humor: 0.3469, Sarcasm: 0.4101, Offensive: 0.4748
Avg F1: 0.4106

Epoch 9/10
Train Loss: 0.7061
Val F1 Scores - Humor: 0.3410, Sarcasm: 0.4129, Offensive: 0.4773
Avg F1: 